# Kontrol Mouse dengan Pelacakan Gerakan Tangan

**Zufar Syaafi'**  
<zufar.syaafie@gmail.com>

---

<div align="center">
    <video width="640" controls>
        <source src="image/demo.mp4" type="video/mp4">
        Your browser does not support the video tag.
    </video>
</div>

In [1]:
# v1.0
# python version: 3.10.18

In [2]:
# # uncomment the following line to install the required packages
# !pip install cv2 mediapipe pyautogui pynput

## 1. Pendahuluan

### 1.1 Latar Belakang

Interaksi manusia dengan komputer pada umumnya dilakukan dengan perantara perangkat input fisik berupa _mouse_ dan _keyyboard_. Namun, apa jadinya jika interaksi itu diimplementasikan dengan metode alternatif yang lebih intuitif berupa gerakan tangan langsung?

### 1.2 Tujuan

Tujuan utama dari proyek ini adalah untuk membagun sebuah program dengan kemampuan sebagai berikut.
1. Mengakses _feed_ video dari _webcam_
2. Mendetksi dan melacak tangan sekaligus jari-jarinya
3. Mengenalis gestur tertentu
4. Menerjemahkan gestur menjadi aksi kontrol mouse tertentu

## 2. Komponen dan Pustaka

Proyek ini menggunakan beberapa pustaka python, yaitu
1. OpenCV (`cv2`): Untuk melakukan _image capturing_ dan _processing_ dari video _webcam_
2. MediaPipe (`mediapipe`): Untuk meng-_handle_ deteksi tangan dan ekstraksi 21 _landmark_ dari tangan yang terdeteksi
3. PyAutoGUI & Pynput: Untuk mengerjakan tugas kontrol mouse secara terprogram
4. NumPy: Untuk membantu melakukan kalkulasi matematis, terutama untuk menghitung jarak dan sudut

In [3]:
import cv2
import mediapipe as mp
import numpy as np
import pyautogui
from pynput.mouse import Button, Controller

## 3. Metodologi dan Cara Kerja

### 3.1 Gestur Tangan

Pengolahan gestur tangan dimulai dengan memberikan 21 _landmark_ untuk telapak tangan. Proyek ini menggunakan standar _landmark_ dari MediaPipe sebagai berikut.

<div align="center">
    <img src="https://mediapipe.dev/images/mobile/hand_landmarks.png" alt="hand_landmarks.png" />
    <p><em>Gambar 1. 21 landmarks tangan.</em></p>
</div>

Adapun gestur yang akan digunakan adalah kondisi-kondisi khusus dari posisi _landmarks_ yang masing-masing secara berurutan mewakili aksi _move_ (memindahkan kursor), stop, klik kiri, klik kanan, klik kanan ganda, dan mengambil tangkapan layar. Lebih detail ada pada _Gambar 2._

<div align="center">
    <table>
        <tr>
            <td align="center"><img src="image/move.png" alt="move.png" /><br><em>Gambar 2.1 Gestur move.</em></td>
            <td align="center"><img src="image/stop.png" alt="stop.png" /><br><em>Gambar 2.2 Gestur stop.</em></td>
            <td align="center"><img src="image/lb.png" alt="lb.png" /><br><em>Gambar 2.3 Gestur klik kiri.</em></td>
        </tr>
        <tr>
            <td align="center"><img src="image/rb.png" alt="rb.png" /><br><em>Gambar 2.4 Gestur klik kanan.</em></td>
            <td align="center"><img src="image/double.png" alt="double.png" /><br><em>Gambar 2.5 Gestur klik kanan ganda.</em></td>
            <td align="center"><img src="image/ss.png" alt="ss.png" /><br><em>Gambar 2.6 Gestur screenshot.</em></td>
        </tr>
    </table>
    <p><em>Gambar 2. Gestur tangan.</em></p>
</div>

Semua gestur pada _Gambar 2._ memiliki aksi statis, kecuali gestur _move_ pada _Gambar 2.1_. Gestur move akan melacak posisi _landmark_ paling ujung dari jari telunjuk (`INDEX_FINGER_TIP`/8) untuk dijadikan posisi kursor di layar komputer. Jadi, kursor akan bergeser mengikuti pergeseran dari _landmark_ terkait.

### 3.2 Perhitungan

Untuk memastikan _mouse_ "virtual" berfungsi diperlukan cara untuk mengidentifikasi kapan jari ditekuk atau diluruskan, atau seberapa dekat satu jari dengan jari lainnya. Proses identifikasi itu bisa dilakukan dengan melakukan perhitungan jarak dan sudut. Ada dua pola utama dalam gestur pada _Gambar 2._. Pertama, ibu jari yang membuka dan menutup. Ini berarti kita menghitung jarak antara titik ujung ibu jari dengan titik akhir dari jari telunjuk atau jarak antara _landmark_ 4 dan 5 (lihat _Gambar 3.1_). Kedua, jari telunjuk atau tengah yang lurus atau menekuk. Ini bisa ditentukan dengan menghitung seberapa besar sudut yang tebentuk antara garis diatas titik tekuk dan dibawah titik tekuk (lihat _Gambar 3.2_).

<div align="center">
    <table>
        <tr>
            <td align="center">
                <img src="image/jarak.png" alt="jarak.png" width="250"><br>
                <em>Gambar 3.1 Jarak antara dua titik.</em>
            </td>
            <td align="center">
                <img src="image/sudut.png" alt="sudut.png" width="250"><br>
                <em>Gambar 3.2 Sudut antara dua garis.</em>
            </td>
        </tr>
    </table>
    <p><em>Gambar 3. Konsep jarak dan sudut.</em></p>
</div>

Untuk menentukan jarak antara dua titik kita bisa lakukan dengan menggunakan persamaan jarak euclidian sebagai berikut.
$$
\text{jarak} = \sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2}
$$
Jadi, misalkan ada dua titik $P_1 = (x_1, y_1)$, dan $P_2 = (x_2, y_2)$, jarak antara dua titik itu bisa dihitung dengan persamaan diatas. Jika hasil jarak kecil, itu artinya ibu jari mendekat ke telunjuk. Berikut implementasi dalam fungsi python.


In [4]:
def distance_between_points(p1, p2):
    """Calculate the distance between two points."""
    return np.linalg.norm(np.array(p1) - np.array(p2))*100

Adapun untuk menentukan sudut yang terbentuk antara dua garis seperti yang sudah dijelaskan, dapat dilakukan operasi sebagai berikut.
$$
\theta = \cos^{-1}\left( \frac{\vec{BA} \cdot \vec{BC}}{\lVert \vec{BA} \rVert \cdot \lVert \vec{BC} \rVert} \right)
$$
Jadi, misalkan diketahui titik-titik:  
  - $A = \text{titik atas}$  
  - $B = \text{titik tekuk}$  
  - $C = \text{titik bawah}$

sudut akan dihitung di titik $B$ menggunakan persamaan di atas. Apabila sudut mendekati 180° dapat disimpulkan jari lurus. Namun, apabila sudut mendekati 90° dapat dikatakan bahwa jari sedang menekuk. Berikut implementasi dalam fungsi python.

In [5]:
def angle_between_points(p1, p2, p3):
    """Calculate the angle between three points."""
    v1 = np.array(p1) - np.array(p2)
    v2 = np.array(p3) - np.array(p2)
    v1_norm = np.linalg.norm(v1)
    v2_norm = np.linalg.norm(v2)

    if v1_norm == 0 or v2_norm == 0:
        return 0
    cos_angle = np.degrees(np.arccos
                           (np.clip
                            (np.dot(v1, v2) / (v1_norm * v2_norm), -1.0, 1.0)))
    return cos_angle

### 3.3 Implementsi Proses

#### 3.3.1 Inisialisasi

Pada tahap ini, semua komponen yang diperlukan untuk fungsionalitas _mouse_ virtual disiapkan. Pertama, koneksi ke _webcam_ diaktifkan melalui perintah `cv2.VideoCapture(0)` untuk menangkap input visual secara _real-time_. Selanjutnya, model `Hands` dari MediaPipe diinisialisasi dengan konfigurasi untuk mendeteksi satu tangan, diatur dengan tingkat kepercayaan deteksi dan pelacakan yang tinggi untuk memastikan akurasi. Terakhir, untuk menerjemahkan gerakan tangan menjadi gerakan kursor, ukuran layar dideteksi menggunakan `pyautogui.size()`, sementara modul kontroler dari `pynput` digunakan untuk mengendalikan _mouse_ secara virtual di seluruh sistem operasi.

In [6]:
# Set pyautogui to fail-safe mode
pyautogui.FAILSAFE = False

# Mouse controller
mouse = Controller()

# Initialize screen dimensions
screen_width, screen_height = pyautogui.size()

# Initialize MediaPipe Hands
mpHands = mp.solutions.hands
hands = mpHands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.8,
    min_tracking_confidence=0.8,
    model_complexity=1
)

draw = mp.solutions.drawing_utils

#### 3.3.2 Operasional Gerakan _Mouse_

Pada tahap ini program berpusat pada dua fungsi utama untuk menerjemahkan gerakan menjadi aksi. Pertama, fungsi `find_finger_tip(result)` berperan sebagai pencari data yang memeriksa hasil pemrosesan dari MediaPipe. Jika sebuah tangan terdeteksi, fungsi ini akan mengekstrak dan mengembalikan lokasi _landmark_ dari ujung jari telunjuk (`INDEX_FINGER_TIP`/8), yang bertindak sebagai titik acuan untuk kursor. Kemudian, jika titik acuan ini berhasil ditemukan, fungsi `move_mouse(finger_tip)` mengambil alih tugas sebagai eksekutor. Fungsi ini mengubah koordinat relatif (nilai 0.0 hingga 1.0) dari ujung jari menjadi koordinat piksel absolut pada layar, sekaligus menerapkan faktor sensitivitas untuk membuat gerakan terasa lebih luas dan nyaman. Terakhir, perintah `pyautogui.moveTo(x, y)` akan dieksekusi untuk memindahkan kursor _mouse_ secara asli di layar.

In [7]:
def find_finger_tip(result):
    if result.multi_hand_landmarks:
        hand_landmarks = result.multi_hand_landmarks[0]
        return hand_landmarks.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP]
    return None


def move_mouse(finger_tip):
    if finger_tip:
        x = int(finger_tip.x * screen_width * 1.5)
        y = int(finger_tip.y * screen_height * 1.5)
        pyautogui.moveTo(x, y)

#### 3.3.3 Kondisi Aksi

Bagian ini menjelaskan logika dari setiap aksi yang dipicu berdasarkan kombinasi antara sudut antar sendi jari dan jarak antara ibu jari dan jari telunjuk (`thumb_dist`). Penggunaan sudut membantu mendeteksi apakah jari dalam keadaan lurus atau menekuk, sementara jarak antara ibu jari dan telunjuk menjadi indikator untuk gestur membuka atau menutup.

Fungsi `is_move_mouse(...)` mengidentifikasi kondisi untuk menggerakkan kursor. Aksi ini dipicu ketika jari telunjuk berada dalam kondisi lurus (sudut antara titik 5-6-8 lebih dari 90 derajat), dan ibu jari dalam keadaan menutup ke arah telunjuk (jarak antara titik 4 dan 5 kurang dari 10). Kombinasi ini biasanya terjadi ketika pengguna menunjuk lurus sambil menutup ibu jari, meniru gerakan natural mengarahkan kursor.

In [8]:
def is_move_mouse(list_landmarks, thumb_dist):
    return (
        angle_between_points(list_landmarks[5], list_landmarks[6], list_landmarks[8])
        > 90
        and thumb_dist < 10
    )

Fungsi `is_left_click(...)` digunakan untuk mendeteksi klik kiri. Syaratnya adalah jari telunjuk menekuk (sudut < 90°), jari tengah lurus (sudut > 90°), dan ibu jari membuka (jarak > 10). Posisi ini menyerupai gestur menekan tombol menggunakan telunjuk sementara jari tengah tetap lurus.

In [9]:
def is_left_click(list_landmarks, thumb_dist):
    return (
        angle_between_points(list_landmarks[5], list_landmarks[6], list_landmarks[8])
        < 90
        and angle_between_points(
            list_landmarks[12], list_landmarks[10], list_landmarks[9]
        )
        > 90
        and thumb_dist > 10
    )

Fungsi `is_right_click(...)` akan mengembalikan nilai benar jika jari telunjuk lurus (sudut > 90°), jari tengah menekuk (sudut < 90°), dan ibu jari membuka (jarak > 10). Kombinasi ini menunjukkan bahwa pengguna menggunakan jari tengah untuk memberi sinyal klik, dengan telunjuk tetap lurus.

In [10]:
def is_right_click(list_landmarks, thumb_dist):
    return (
        angle_between_points(list_landmarks[5], list_landmarks[6], list_landmarks[8])
        > 90
        and angle_between_points(
            list_landmarks[12], list_landmarks[10], list_landmarks[9]
        )
        < 90
        and thumb_dist > 10
    )

Fungsi `is_double_click(...)` mendeteksi klik ganda. Syaratnya adalah baik jari telunjuk maupun jari tengah berada dalam kondisi menekuk (sudut < 90°) dan ibu jari membuka (jarak > 10). Ini menyerupai gestur menekan dua kali cepat menggunakan telunjuk.

In [11]:
def is_double_click(list_landmarks, thumb_dist):
    return(
        angle_between_points(
            list_landmarks[5],
            list_landmarks[6],
            list_landmarks[8]) < 90 and
        angle_between_points(
            list_landmarks[12],
            list_landmarks[10],
            list_landmarks[9]) < 90 and
        thumb_dist > 10
    )

Fungsi `is_screenshot(...)` mengembalikan nilai benar jika kedua jari (telunjuk dan tengah) menekuk (sudut < 90°) dan ibu jari dalam posisi menutup (jarak < 10). Gestur ini bisa diinterpretasikan sebagai tanda menutup atau menggenggam yang bisa digunakan sebagai pemicu pengambilan tangkapan layar.

In [12]:
def is_screenshot(list_landmarks, thumb_dist):
    return (
        angle_between_points(list_landmarks[5], list_landmarks[6], list_landmarks[8])
        < 90
        and angle_between_points(
            list_landmarks[12], list_landmarks[10], list_landmarks[9]
        )
        < 90
        and thumb_dist < 10
    )

### 3.3.4 Deteksi Gestur

Fungsi `detect_gesture(...)` bertanggung jawab untuk mengenali gestur tangan berdasarkan landmark jari yang terdeteksi, lalu menjalankan aksi tertentu seperti menggerakkan kursor, melakukan klik, atau mengambil tangkapan layar. Proses ini dilakukan secara real-time dengan mengambil input berupa `frame` dari kamera, daftar `list_landmarks`, dan hasil pendeteksian dari model.

Langkah pertama dalam fungsi ini adalah memeriksa apakah jumlah landmark yang terdeteksi cukup (minimal 21 titik). Selanjutnya, sistem mencari koordinat ujung jari telunjuk (`index_finger_tip`) menggunakan fungsi `find_finger_tip(...)` dan menghitung jarak antara ibu jari dan pangkal jari telunjuk (`thumb_dist`) menggunakan fungsi `distance_between_points(...)`.

Setelah informasi jarak dan posisi terkumpul, sistem akan memeriksa satu per satu kondisi gestur:

- Jika kondisi `is_move_mouse(...)` terpenuhi (telunjuk lurus dan ibu jari menutup), maka kursor akan digerakkan ke posisi ujung jari telunjuk.
- Jika `is_left_click(...)` terpenuhi (telunjuk menekuk, tengah lurus, ibu jari membuka), maka sistem melakukan klik kiri dan menampilkan teks "Left Click" di layar.
- Jika `is_right_click(...)` terpenuhi (telunjuk lurus, tengah menekuk, ibu jari membuka), maka sistem melakukan klik kanan dan menampilkan "Right Click".
- Jika `is_double_click(...)` aktif (telunjuk dan tengah menekuk, ibu jari membuka), maka sistem melakukan klik ganda.
- Jika `is_screenshot(...)` aktif (telunjuk dan tengah menekuk, ibu jari menutup), maka sistem mengambil tangkapan layar dan menyimpannya sebagai file `"screenshot.png"`.

Setiap aksi juga disertai teks indikator yang ditampilkan di sudut atas frame menggunakan `cv2.putText(...)`, dengan warna yang berbeda-beda untuk membedakan jenis aksi.

In [13]:
def detect_gesture(frame, list_landmarks, result):
    if len(list_landmarks) >= 21:
        index_finger_tip = find_finger_tip(result)
        thumb_dist = distance_between_points(list_landmarks[4], list_landmarks[5])

        if is_move_mouse(list_landmarks, thumb_dist):
            move_mouse(index_finger_tip)
            cv2.putText(frame, "Move Mouse", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        elif is_left_click(list_landmarks, thumb_dist):
            mouse.click(Button.left, 1)
            # add a small delay to allow the system to process
            cv2.waitKey(500)
            cv2.putText(frame, "Left Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        elif is_right_click(list_landmarks, thumb_dist):
            mouse.click(Button.right, 1)
            # add a small delay to allow the system to process
            cv2.waitKey(500)
            cv2.putText(frame, "Right Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        elif is_double_click(list_landmarks, thumb_dist):
            mouse.click(Button.left, 2)
            # add a small delay to allow the system to process
            cv2.waitKey(500)
            cv2.putText(frame, "Double Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        elif is_screenshot(list_landmarks, thumb_dist):
            screenshot = pyautogui.screenshot()
            screenshot.save("screenshot.png")
            cv2.putText(frame, "Screenshot Taken", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2)

#### 3.3.5 Pengambilan dan Pemrosesan Video Kamera

Fungsi `camera()` merupakan komponen utama untuk mengakses _webcam_, memproses citra tangan secara _real-time_, dan menjalankan deteksi gestur yang telah didefinisikan sebelumnya.

Pertama, kamera diakses menggunakan `cv2.VideoCapture(0)` yang membuka perangkat kamera utama (biasanya webcam). Proses pembacaan frame dilakukan secara terus-menerus selama kamera terbuka. Setiap frame akan dibalik secara horizontal menggunakan `cv2.flip(frame, 1)` agar sesuai dengan orientasi gerakan pengguna (efek cermin), lalu dikonversi dari format BGR ke RGB agar dapat diproses oleh MediaPipe Hands.

Selanjutnya, sistem mempersiapkan list kosong `list_landmarks` untuk menyimpan koordinat dari titik-titik landmark tangan yang terdeteksi. Kemudian, citra RGB diproses oleh `hands.process(...)` dari MediaPipe. Jika tangan berhasil dideteksi, maka landmark tangan pertama (`multi_hand_landmarks[0]`) akan digambar pada frame menggunakan `draw.draw_landmarks(...)`, dan setiap titik landmark (berupa nilai `x` dan `y`) akan disimpan dalam `list_landmarks`.

Fungsi `detect_gesture(...)` kemudian dipanggil, mengirimkan frame saat ini, daftar landmark, dan hasil deteksi sebagai argumen, untuk menentukan apakah ada gestur tertentu yang dikenali (seperti klik, gerak mouse, screenshot, dll).

Akhirnya, frame yang sudah diproses ditampilkan melalui `cv2.imshow(...)`. Pengguna bisa keluar dari aplikasi dengan menekan tombol `Esc`. Saat keluar, kamera dan semua jendela OpenCV ditutup dengan aman menggunakan `cap.release()` dan `cv2.destroyAllWindows()`.

In [14]:
def camera():
    cap = cv2.VideoCapture(0)
    
    try: 
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                print("Failed to grab frame")
                break
            
            # Convert the BGR image to RGB
            frame = cv2.flip(frame, 1)
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # List to hold hand landmarks
            list_landmarks = []

            # Process the frame with MediaPipe Hands
            results = hands.process(rgb_frame)
            if results.multi_hand_landmarks:
                hand_landmarks = results.multi_hand_landmarks[0]
                draw.draw_landmarks(frame, hand_landmarks, mpHands.HAND_CONNECTIONS)
                for landmark in hand_landmarks.landmark:
                    list_landmarks.append((landmark.x, landmark.y))    

            detect_gesture(frame, list_landmarks, results)    

            # show the frame
            cv2.imshow('Camera Feed', frame)
            # Exit on 'esc' key press
            if cv2.waitKey(1) & 0xFF == 27:
                break

    finally:
        cap.release()
        cv2.destroyAllWindows()

In [15]:
# Run the camera function to start the application
camera()

# Thank You! ^^

Terima kasih telah membaca hingga akhir 
Semoga proyek ini bisa menginspirasi dan membantu!

---